Written by:
- Joshua Wylie

Edited by:
- Witold Nazarewicz

# Introduction

In this notebook, we'll look at different methods you can use to read-in data from a file. This will not be an exhaustive list of methods, but hopefully a nice introduction that will be helpful in your courses and beyond! Additionally, after we practice reading-in data, we will look at helpful packages for plotting the read data in 2D and 3D formats.

If at any point you are confused by the process we are doing, please go to the [ASCSN GitHub coding page](https://github.com/ascsn/online-guides/tree/main/Coding) which has more notebooks that might be a helpful resource!

We will start this demonstration by creating a set of random data using ```numpy``` (which is installed below if not already). This random set of data will be used in the analytic equation
$$
y = 6 \sin\left(\frac{x}{3}\right).
$$
This will be our true function to which we will add in additional error.

If you haven't already installed ```numpy```, the following lines of code might do it for you! If these don't work on your local machine, you unfortunately might have to do a bit of work handling the errors. Otherwise you can always try working on [Google Colab](https://colab.research.google.com/) or GitHub Codespace ([see video on using Codespaces](https://www.youtube.com/watch?v=AJFen_Z3mWM)).

NOTE: If you run the following without pip installed, you will get an error. If that's the case, look at installing pip on your machine or again consider using the available online tools above.

In [ ]:
!pip3 install numpy

In [2]:
# Import numpy for use
import numpy as np

# If you're confused or unsure about packages, see the following for an intro to packages!
# https://github.com/ascsn/online-guides/blob/main/Coding/Python%20Packages.ipynb

In [3]:
# Define our true function
def true_function_A(x):
    return 6*np.sin(x/3)

In [4]:
# Set a seed for a consistent pseudo-random number generator
# this ensures we will always get the same random numbers regardless of run instance
np.random.seed(981)

# Set number of data points to use
number_of_data_points = 11

# Set an array of x values
data_A_x = np.random.rand(number_of_data_points)

# Set an array of error bars
data_A_errors = np.random.rand(number_of_data_points)

# Create the "y" data
data_A_y = np.random.multivariate_normal(true_function_A(data_A_x), np.diag(data_A_errors)**2)

Now that we have our ```numpy``` data, we can always print it to view the results. Let's do that now.

In [ ]:
print('A\'s x data', data_A_x)
print() # We can just space the prints out by printing a blank line!
print('A\'s errors', data_A_errors)
print() # We can just space the prints out by printing a blank line!
print('A\'s y data', data_A_y)

Great, but it's a bit hard to read and not easy to reference on-the-fly. Moreover, every time we want to print this data we'd have to use the split arrays and write multiple print statements. We can always group together these numpy arrays into one 2D array, but it would just be a set of numbers with no identifying information. Another thing we can use instead is a *dictionary* which is native to ```python```.

Dictionaries are a structure containing a *key* and an *item*. The *key* is essentially a lookup name or number allowing a user to call a specific element without having to keep track of numerical indices. The *item* can be anything from a single string to a list of strings to a list of floats. We show an example of how to define a dictionary and how to call a specific attribute.

In [ ]:
# Create a dictionary
sample_dict = dict(
    name=['Aang', 'Bumi', 'Cabbage Merchant'],
    age=[112, 112, 40]
)

# Print the entire dictionary
print('Total dictionary:')
print(sample_dict)

# Print only the "name" subset
print('\nNames only:', sample_dict['name'])

# Print only the "age" subset
print('\nAges only:', sample_dict['age'])

While dictionaries are nice, they are not always the best for handling data and need to have the data input manually. This is where the package ```pandas``` comes in as it has many very helpful data handling features.

# Using ```pandas```

To start, we'll import the ```pandas``` package.

In [ ]:
!pip3 install pandas

In [10]:
# import pandas which allows us to more easily handle and organize lots of data in python!
import pandas as pd

Just as we did above, we can define a dictionary containing some data.

In [ ]:
# This uses pythons native dictionary type which stores any type of data under some key name
data_dictionary = {
    'x': data_A_x,
    'error': data_A_errors,
    'y': data_A_y
}

print(data_dictionary)

# You can distinguish a dictionary by the name "dict()" or curly braces "{}" similar to
# a "list()" or "[]" format if you're familiar with those.

You'll notice that our arrays are stored under "x", "error", and "y" which can be called to access the corresponding data like we see below.

In [ ]:
print(data_dictionary['error'])

This called the "error" data which we can easily access at any time!

```pandas``` can also handle data with callable names using what are known as DataFrames. We can focus on the DataFrame version now. Thankfully, ```pandas``` makes this really easy for us as we can just pass our dictionary into a DataFrame type and we're done!

In [ ]:
data_df = pd.DataFrame(data_dictionary)

# Use the display function in Jupyter to nicely show a DataFrame
display(data_df)

We can also make a DataFrame from scratch without having to convert it from a dictionary. Before we do this, we'll need to prepare our data a little by merging all 1D ```numpy``` arrays into a 2D array.

In [ ]:
# Merge all 3 1D numpy ararys into one 2D array by stacking the 1D arrays on top of each other
merged_data = np.vstack((data_A_x, data_A_y, data_A_errors))
print('Shape of the merged data =',merged_data.shape)

We notice that we now have a 3 row and 11 column array. We can convert this to a ```pandas``` DataFrame by transposing the data (making a 3 column and 11 row array) and assigning column names to it.

In [ ]:
# Create a DataFrame as a list of numpy arrays with set column names
data_df = pd.DataFrame(merged_data.T, columns=['x', 'y', 'error'])

display(data_df)

We can use a few nice features like built-in sorting to make the above DataFrame nicer.

In [ ]:
# Sort by column x in increasing order
data_df = data_df.sort_values('x')

display(data_df)

Something nice about any stored data types like those in ```numpy``` and ```pandas``` is that they can be easily saved to a text, csv, or other data file. We can consider an example of this below for the above data.

## Reading and Writing to a file

We will focus on a text case below. We will use ```numpy```'s built-in random number generator to generate a set of values pulled from a Gaussian and a Poisson distribution.

In [21]:
# Generate the random data using the code below
points=200
example_out_df = np.linspace(-1,1,points)
example_out_df = pd.DataFrame({
    'Gaussian':np.random.normal(size=points),
    'Poisson':np.random.poisson(1, size=points),
})

Before we highlight the great features in ```pandas```, we will first show how one might go about saving the above data using native ```python``` functions. This will be a bit more cumbersome and hopefully give a good appreciation for ```pandas```.

In [ ]:
# To write to any file, we need to tell python to first open the file we wish to write to
file_name = 'simple_output_file_example.csv'

# We use the "with" command to tell python to execute everything inside and then auto-close the file afterwards
# ensuring that we do not have any issues of not closing or writing unwanted information later
with open(file_name, 'w') as file: # Open the file under file_name
    # Any operations now on the "file" object will be saved in the file_name we want
    # We will loop through the data row by row and write the data
    for i in range(len(example_out_df)):
        # Access the i-th row and all columns to be saved to row_data as a list
        row_data = example_out_df.iloc[i,:].to_list()

        # Convert each list element into a string format from float format
        str_list = [str(rd) for rd in row_data] # Iterate via a list comprehension (see below for details)

        # Merge the row_data from an iterable (list-like) format into a string separated by a comma
        str_data = ','.join(str_list)

        # write the string of data with a new line character "\n" to start a new line
        file.write(str_data+'\n')


# BONUS: List comprehension as a tool
# Python allows for what is known as a list comprehension or essentially a way to make a for-loop more compact!
# We give an example here of a small dataset which we can use either a for loop or list comprehension to do the same thing.
# List comprehensions are sometimes faster, so it can be nice to use them!

# Define a dataset
list_comp_data = [3.4, 52.15, 346.946] # We define 3 random floats we will operate on

# Our goal will be to convert this data to a string formatted list and print said list
# We first do our loop version
for_loop_list = [] # Create an empty list to add data to

# Iterate through each list element and get the data
for comp_data in list_comp_data: # By doing "_item_ in _list_" we go through each item in index order in the list
    for_loop_list.append(str(comp_data)) # Append the string-converted float to the list

# Now we convert the data to a list of strings using a list comprehension
list_comp_list = [str(comp_data) for comp_data in list_comp_data]

# Print both versions for comparison
print('Converted list using a for loop           :', for_loop_list)
print('Converted list using a list comprehension :', for_loop_list)

If you open the file named ```simple_output_file_example.csv``` you should see all of the data in a comma-separated values (csv) format where all of the data in a row is separated by commas. As you can see above, ```python``` can allow you to write almost anything to a file using the ```with open()``` function. The drawback to using native python is that it requires a bit of effort. This is where ```pandas``` comes in since it can nicely save the DataFrames as we'll show below!

In [40]:
# Save the data to a csv using pandas
example_out_df.to_csv('simple_output_file_example.csv', index=None)

You should now see a file with the name "simple_output_file_example.csv" in the directory you are running this notebook in. If you open it, you'll find the above data is written!

We also would like to point out that ```pandas``` does have some rudimentary plotting features like a histogram function (shown below). If we want to visualize data in a specific way, there are other libraries which are better suited to your needs. We show a histogram of the above generated data plotted through the available tools in Pandas.

In [ ]:
# We can quickly plot a histogram of a particular dataset
hist_bins = 25

# Plot Gaussian data
example_out_df['Gaussian'].plot.hist(bins=hist_bins)

# Plot Poisson data
# example_out_df['Poisson'].plot.hist(bins=hist_bins)

We can also practice reading data from a given data file. We provide an example below which shows how to read-in a data file with native ```python``` and ```pandas```. We note that sometimes files are not nicely structured and may not be easily readable or have holes in it. Our example below illustrates a smaller random data set but with additional holes in it to produce Not-a-number (NaN) errors in the missing spaces.

In [ ]:
# Native python version
read_file = 'simple_read_file_with_holes_example.csv'

# we use the same "with open" command but this time we set the type to read 'r'
with open(read_file, 'r') as file:
    # If we just want a list of strings, we can have python read all lines at once and store each one in a list
    list_of_lines = file.readlines()

# We can view the list of lines
print(list_of_lines)

As we see in the above list, we have all of the data in one string. We would need to parse this manually to get the data out. There's also a pesky swarm of new-line characters ```\n``` which appear in each string we'll need to get rid of too! In order to have a nice file format type, we will iterate through each list item and save it as a list of lists (a 2D list). We can always convert this to a ```numpy``` array later!

In [ ]:
data_list = [] # Create an empty list to store our data as we parse it

# iterate through each list string item
for line in list_of_lines:
    # Remove the new-line character from the string
    no_new_line = line.replace('\n','')

    # Split up our no_new_line string into a list based on the comma ',' delimiter
    list_of_line_data = no_new_line.split(',')

    # Our data is still in string format, so we'll need to iterate over each element and convert it into a float
    # We do so with a list comprehension which includes an if-else statement to catch an error for when we have something
    # which cannot be converted to a float e.g. '' has no numeric value and we assign NaN
    list_of_line_data = [
        float(dat) if (dat != '') else np.nan for dat in list_of_line_data
        ]

    # Append this list to our data_list to make a 2D list of lists
    data_list.append(list_of_line_data)

# Convert our list of lists to a 2D numpy array
numpy_data = np.array(data_list)
# Print our data
print(numpy_data)

That was kind of a mess and annoying to handle the gaps in the data. ```pandas``` can handle some of this nicely for us with one simple command!

In [ ]:
# We can read the csv and use the setting "header=None" to tell pandas that it should use all values
# in the file. If we do not use "header=None" then the first line will serve as the column names.
example_read_df = pd.read_csv('simple_read_file_with_holes_example.csv', header=None)
display(example_read_df)

The above data would appear to be missing in some segments, but in reality this is because there are extra commas in the file. Think back to above where we needed the if-else statement to handle the empty list elements (```''```) and replaced them with ```NaN```.  ```pandas``` recognizes the commas as specific data and thinks there are more columns than there actually are. If you're curious, you can certainly take a look!

This is all well and good, but we can't really get a good idea of what we're looking at unless we know the data can be nicely plotted using the simple features above. Let's use ```matplotlib``` to visualize this!

# Using Matplotlib

In [ ]:
!pip3 install matplotlib

In [52]:
# import matplotlib which is the go-to package for simple plots
import matplotlib.pyplot as plt

```matplotlib``` is a library for visualizing provided information. It works by displaying objects, sometimes called "assets," on a "figure" panel. There are many things we can do on this figure, but we'll start simple for now. 

```matplotlib``` has tried to make it easy to plot things like scatter plots. We'll use these features to plot our data.

In [ ]:
# To start, we'll make a figure and a subplot panel overlaying it
fig, ax = plt.subplots(figsize=[5,5]) # Figure and axis subplot panel respectively

# We will plot data directly onto the axis panel
ax.errorbar(
    data_df['x'], data_df['y'], yerr=2*data_df['error'], # plot x, y, and error data
    fmt='o', capsize=7 # set the format to be circular data points and error bar width size
)

# Set the x and y-labels for the plot
ax.set_xlabel('x',fontsize=16)
ax.set_ylabel('y',fontsize=16)

# Set x and y limits
ax.set_xlim(0, 1)
ax.set_ylim(-2,4)

# Set title
ax.set_title('Example data plot', fontsize=16)

Now that we have some data, we might also have a model we wish to plot with respect to. For example, we might want to try to fit a straight line to the data since we may not know the exact analytic form of the data. We show how one might go about plotting a line on the same plot.

In [ ]:
# Use the same code as before...
fig, ax = plt.subplots(figsize=[5,5]) # Figure and axis subplot panel respectively

# We will plot data directly onto the axis panel
ax.errorbar(
    data_df['x'], data_df['y'], yerr=2*data_df['error'], # plot x, y, and error data
    fmt='o', capsize=7, # set the format to be circular data points and error bar width size
    label='Data' # Note that we now include a label to go with our legend later
)

# Set the x and y-labels for the plot
ax.set_xlabel('x',fontsize=16)
ax.set_ylabel('y',fontsize=16)

# Set x and y limits
ax.set_xlim(0, 1)
ax.set_ylim(-2,4)

# Set title
ax.set_title('Example data plot', fontsize=16)

# Now we can create any arbitrary line and plot it
line = 2 * data_df['x'] - 0.25
ax.plot(data_df['x'], line, label='Example Line')

# Include a legend on the given axis to the top left quadrant
ax.legend(loc='upper left')

# We can also save any matplotlib figure to a file type by the following
plt.savefig('example_figure_random_linear_data.pdf')

```matplotlib``` can also plot 3D objects. We show an example case below of plotting a Riemann Surface for a $\sqrt{z}$ complex function. You do not need to know anything about this function, but we just show it as an example on how to plot 3D functions including complex-valued functions!

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Define the complex function sqrt(z)
def sqrt_complex(z, branch):
    if branch == 0:
        return np.sqrt(z)
    else:
        return -np.sqrt(z)

# Create a grid of complex numbers for plotting
x = np.linspace(-2, 2, 400)
y = np.linspace(-2, 2, 400)
X, Y = np.meshgrid(x, y)
Z = X + 1j * Y

# Apply the sqrt function to each element of the grid for both branches
W_pos = sqrt_complex(Z, branch=0)
W_neg = sqrt_complex(Z, branch=1)

# Create the plot
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

# Plot the positive branch
ax1.plot_surface(X, Y, np.angle(W_pos), facecolors=plt.cm.Blues(np.abs(W_pos)),
                 rstride=5, cstride=5, alpha=0.8)
ax1.set_title("Positive Branch")
ax1.set_xlabel("Re(z)")
ax1.set_ylabel("Im(z)")
ax1.set_zlabel("|sqrt(z)|")

# Plot the negative branch
ax2.plot_surface(X, Y, np.angle(W_neg), facecolors=plt.cm.Reds(np.abs(W_neg)),
                 rstride=5, cstride=5, alpha=0.8)
ax2.set_title("Negative Branch")
ax2.set_xlabel("Re(z)")
ax2.set_ylabel("Im(z)")
ax2.set_zlabel("|sqrt(z)|")

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


## Alternate plotting method: ```plotly```

In addition to ```matplotlib``` there exists a second plotting package called ```plotly```. This has many useful features including more user-interactivity. We show the same function but plotted using ```plotly``` below. A key thing to note is ```plotly``` images tend to be a bit more resource intensive and will be much larger in file size.

In [ ]:
!pip3 install plotly

In [ ]:
# Import dependencies
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

In [ ]:
# Define the complex function sqrt(z)
def sqrt_complex(z, branch):
    if branch == 0:
        return np.sqrt(z)
    else:
        return -np.sqrt(z)

# Create a grid of complex numbers for plotting
x = np.linspace(-2, 2, 400)
y = np.linspace(-2, 2, 400)
X, Y = np.meshgrid(x, y)
Z = X + 1j * Y

# Apply the sqrt function to each element of the grid for both branches
W_pos = sqrt_complex(Z, branch=0)
W_neg = sqrt_complex(Z, branch=1)

# Create the Riemann surface plot with both branches
fig = go.Figure()

# Plot the positive branch in blue
fig.add_trace(go.Surface(z=np.angle(W_pos), surfacecolor=np.abs(W_pos), cmin=0, cmax=2*np.pi, colorscale='Blues', name='Positive Branch'))

# Plot the negative branch in red
fig.add_trace(go.Surface(z=np.angle(W_neg), surfacecolor=np.abs(W_neg), cmin=0, cmax=2*np.pi, colorscale='Reds', name='Negative Branch'))

# Configure the layout
fig.update_layout(scene=dict(zaxis=dict(range=[0, 2*np.pi])), title='Riemann Surface for sqrt(z)')

# Show the combined plot
fig.show()

# Assignment

Before we end this notebook, we'll provide a little assignment which will be be used in the next notebook for $\chi^2$ minimization. Please save the above dataset with errors (```data_df```) to a csv using ```pandas``` in the cell below.

In [26]:
# Save the dataset using Pandas in this cell
output_file_name = 'assignment_output_for_reading_data_and_plotting.csv' # Use this name for the output file

# Your code goes below here...


# Conclusions

Now that we've gone through an introduction to ```pandas``` and ```matplotlib```, we hope this will be useful in the next homework which will need you to consider a similar problem as our line example, but perform minimization of a model on that data.